Here, we want to know the model with the best explanatory power, compared to the rest.

## Requirements

```bash

mamba create -n acnets_bambi
mamba activate acnets_bambi
mamba install bambi ipykernel seaborn watermark
```

In [1]:
import bambi
import arviz as az
import xarray as xr
import re
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
INPUT_MODEL_NAME = 'connectivities_classifier-SVML1_measure-accuracy_top-10_cv-100x4fold'
OUTPUT_MODEL_NAME = f'{INPUT_MODEL_NAME}_bambi-1000draws'

In [3]:
results = xr.open_dataset(f'models/{INPUT_MODEL_NAME}.nc').load()

split_score_cols = [c for c in list(results.keys()) if re.match(r'split\d+_test_score', c)]
grid_cv_results = results[split_score_cols + ['rank_test_score', 'mean_test_score']].to_pandas()
grid_cv_results.index.name = 'label'
grid_cv_results = grid_cv_results.reset_index()

scores = grid_cv_results
# scores['label'] = scores['params'].apply(lambda p: _create_label(list(p.values())))
split_score_cols = [c for c in scores.columns if 'split' in c]
scores['score'] = scores[split_score_cols].apply(lambda x: list(x), axis=1)
scores = scores.explode('score').reset_index(drop=True)
scores = scores.sort_values('rank_test_score', ascending=False)
scores['label'] = scores['label'].apply(lambda x: x.replace('partial correlation', 'partial_correlation'))

scores[['atlas','connectivity_metric']] = scores['label'].apply(lambda x: x.split(' ')).apply(pd.Series)
scores = scores[['atlas', 'connectivity_metric', 'score']]

# proportion of correct predictions
scores['score'] = scores['score'] * 100

def cleanup_label(label):
    label = label.replace('dosenbach2010', 'Dosenbach2010')
    label = label.replace('gordon2014_2mm', 'Gordon2014')
    label = label.replace('difumo_64_2mm', 'DiFuMo64')
    label = label.replace(', ',' : ')
    return label

scores['connectivity_metric'] = scores['connectivity_metric'].apply(cleanup_label)
scores['atlas'] = scores['atlas'].apply(cleanup_label)

scores.columns = ['P', 'C', 'y']


In [4]:

# metric_cats=['covariance', 'tangent', 'correlation', 'partial_correlation','precision']
# atlas_cats=["DiFuMo64", "Dosenbach2010", "Gordon2014"]
# scores['atlas2'] = scores['atlas'].astype('category').cat.reorder_categories(atlas_cats)
# scores['connectivity_metric2'] = scores['connectivity_metric'].astype('category').cat.reorder_categories(metric_cats)

# scores['atlas2'] = scores['atlas2'].cat.as_ordered()
# scores['connectivity_metric2'] = scores['connectivity_metric2'].cat.as_ordered()

In [5]:
model = bambi.Model('y ~ P * C', scores, family='gaussian', auto_scale=True)
model.build()
model.graph('plain_with_params', name='outputs/figures/bayesian_model', dpi=300, fmt='png')


/home/morteza/mambaforge/envs/acnets_bambi/lib/python3.10/site-packages/bambi/models.py:1009: UserWarning: Formattings other than 'plain' are currently not supported.
  graphviz = pm.model_to_graphviz(model=self.backend.model, formatting=formatting)


In [6]:
fitted = model.fit(tune=1000, draws=2000, init='adapt_diag')

fitted.to_netcdf(f'outputs/model_comparison/{OUTPUT_MODEL_NAME}_2.nc')

Auto-assigning NUTS sampler...
Initializing NUTS using adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)


'outputs/model_comparison/connectivities_classifier-SVML1_measure-accuracy_top-10_cv-100x4fold_bambi-1000draws_2.nc'

## Part 2: Visualization

In [ ]:
fitted = az.data.inference_data.InferenceData.from_netcdf(f'outputs/model_comparison/{OUTPUT_MODEL_NAME}.nc')

az.summary(fitted).sort_values('mean', ascending=False).to_csv(f'outputs/model_comparison/summary_{OUTPUT_MODEL_NAME}.csv')

az.summary(fitted).sort_values('mean', ascending=False).to_markdown(f'outputs/model_comparison/summary_{OUTPUT_MODEL_NAME}.md')

az.summary(fitted).sort_values('mean', ascending=False)
# DEBUG plot using arviz
# az.plot_trace(fitted, legend=True)
# plt.tight_layout()
# plt.show()

In [ ]:
posteriors = []
for v in fitted['posterior'].data_vars:
    if 'sigma' in v or 'Intercept' in v:
        continue
    posterior = fitted['posterior'][v].to_dataframe().reset_index()
    posterior['parameter'] = v
    posterior['variable'] = posterior[f'{v}_dim'].apply(cleanup_label)
    posterior['posterior'] = posterior[v]
    posterior = posterior[['parameter', 'variable', 'posterior']].copy()
    posteriors.append(posterior)


posteriors = pd.concat(posteriors)

import seaborn as sns
sns.set('paper', 'ticks', font_scale=1.5, rc={
    'xtick.bottom': True,
    'ytick.left': True,
    'axes.edgecolor': 'k',
    'axes.linewidth': 1.25,
    'axes.grid': True
})

def hdi_errorbar(**kwargs):
    az.summary(fitted, fmt='wide').loc['atlas[difumo_64_2mm]']['hdi_3%']
    print(kwargs)

g = sns.catplot(data=posteriors,
            y='variable',
            x='posterior',
            col='parameter',
            sharey=False,
            join=False,
            sharex=True,
            scale=1.2,
            col_wrap=1,
            color='k',
            errorbar=('sd', 2), capsize=0,
            kind='point',
            height=3, aspect=3,
)

g.axes[0].yaxis.grid(True)
g.axes[1].yaxis.grid(True)
g.axes[2].yaxis.grid(True)

# g.set_xticklabels(fontsize=10)
# g.set_titles(template='{col_name}')

g.axes[0].set_title(r'$P$ (atlas)', fontweight='bold')
g.axes[1].set_title('$C$ (connectivity metric)', fontweight='bold')
g.axes[2].set_title('$P:C$ (interaction between atlas and connectivity metric)', fontweight='bold')

# , baseline = DiFuMo64)

g.set_axis_labels(r'posterior (i.e., coefficient in $y \sim P + C + P:C$)', '', fontsize=10)
plt.tight_layout()
plt.savefig(f'outputs/figures/model_comparison', dpi=300, bbox_inches='tight')
plt.show()

<Figure size 900x900 with 3 Axes>

In [9]:
# posterior_predictive = model.predict(fitted, kind="pps")
# az.plot_ppc(fitted)


In [10]:
%reload_ext watermark
%watermark -n -u -v -iv -w